# Scraping coinmarketcap for ICO prices
Leon Yin | Github: [yinleon](#TODO) | Twitter: [@leonyin](#TODO) | Updated: 2017-11-12

This is notebook that explains how to make a scraper that collects a [table of ICO stats](https://coinmarketcap.com/all/views/all/) from the site coinmarketcap.com. 

While we're at it we create a few helpful metadata columns, makes numerical values machine-readible, and perform some simple data analysis. Because this is a Jupyter Notebook you can run it on your own machine :)


View this on Github [here](#TODO) or NBViewer [here](#TODO).<br>
The scraper is available as Python script [here](#TODO).<br>
The hourly data is avilable open source on Amazon s3 [here](#TODO)

If you like this project please help support it by contributing time to make it better, or donating to help [pay for hosting](#TODO).

### Table of Contents
1. [Scraping Data with Requests and Beautiful Soup](#scrape)
2. [Cleaning Data with Pandas](#clean)
3. [Analysis with Pandas](#analysis)
4. [Next Steps](#next-steps)

In [1]:
import os
import re
import datetime
import requests

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
root_dir = '../'
table_url = 'https://coinmarketcap.com/all/views/all/'
table_id = 'currencies-all'
today = datetime.datetime.now()

## Scraping a Website with Requests and Beautiful Soup <a id='scrape'></a>
Let's visit the coinmarketcap website programatically using the requests package...

In [3]:
r = requests.get(table_url)

Among other things, `r` contains the html content of the page we visited.

In [4]:
r.content[:300]

b'<!DOCTYPE html>\n<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->\n<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8"> <![endif]-->\n<!--[if IE 8]>         <html class="no-js lt-ie9"> <![endif]-->\n<!--[if gt IE 8]><!--> \n<html class="no-js" lang="en"> <!--<![endif]-->'

BeautifulSoup is the defacto package (still?) for parsing HTML content.<br>
We can send the html from `r` into a parsable object.

In [5]:
soup = BeautifulSoup(r.content, 'lxml')

This new object (`soup`) comes in handy because we can isolate sections of the HTML page using `soup.find()`.<br>
The seciton we are after is the element that contains the ICO data.<br>
We can use the `inspect element` feature from Chrome to identify the ID (in this case currencies-all) of this table.

<img src='../media/find_element.png'></img>

I stored the ID for this table as a variable `table_id`.

In [6]:
table_id

'currencies-all'

Let's isolate the table element using `table_id`, and read it into a Pandas dataframe.

In [7]:
html_tbl = str(soup.find('table',{'id': table_id}))
df = pd.read_html(html_tbl, index_col=0)[0]

We can get a peak at the top 5 ICOs.

In [8]:
df.head()

,Name,Symbol,Market Cap,Price,Circulating Supply,Volume (24h),% 1h,% 24h,% 7d
#,,,,,,,,,
1,BTC Bitcoin,BTC,"$109,729,678,760",$6579.45,16677637,"$6,825,140,000",0.14%,5.64%,-9.47%
2,ETH Ethereum,ETH,"$30,201,865,132",$315.58,95702116,"$1,369,980,000",-0.11%,3.22%,4.57%
3,BCH Bitcoin Cash,BCH,"$21,126,209,247",$1257.49,16800300,"$4,594,130,000",7.16%,-18.35%,100.02%
4,XRP Ripple,XRP,"$7,783,447,925",$0.202002,"38,531,538,922 *","$160,219,000",-0.49%,1.67%,-2.11%
5,LTC Litecoin,LTC,"$3,273,470,039",$60.83,53811557,"$299,017,000",-0.58%,2.94%,9.49%


Now that we have everything in Pandas, we can do some extra janitorial work, and analysis.<br>
If you've never used Pandas, you're in for a treat!

## Cleaning Data with Pandas <a id='clean'></a>
The data we have is straight from the HTML table--<br>
it is human-readible, but not machine readible.

Let's use Pandas to clean up the the data (stored in a DataFrame `df`)...<br>
As a first step, we can add a timestamp for context.

In [9]:
df['scrape_timestamp'] = today

### Renaming Columns
Let's make the columns more descriptive, by including the unit in each column (USD).<br>
We can do this by replacing column names using a key-value store (a dictionary)

In [10]:
col_name_w_currency = {
    'Market Cap' : 'market_cap_usd',
    'Price' : 'price_usd',
    'Volume (24h)': 'volume_24h_usd',
}

and a function that operates on each column name.

In [11]:
def clean_up_col(col):
    '''
    Adds currency unit to relevant column names,
    replaces spaces for underscores, 
    replaces % symbols for "percent_change",
    and returns the updated column in lower case.
    '''
    col = col_name_w_currency.get(col, col)
    col = col.replace(' ', '_')
    col = col.replace('%', 'percent_change')
    return col.lower()

Now let's iterate through each column name in `df`, and apply `clean_up_col` to each.<br>
For reference: `[x for x in some_iterator]` is called a list comprehension, which is a slight modification to a for loop.

In [12]:
df.columns = [clean_up_col(c) for c in df.columns]

In [13]:
df.dtypes

name                          object
symbol                        object
market_cap_usd                object
price_usd                     object
circulating_supply            object
volume_24h_usd                object
percent_change_1h             object
percent_change_24h            object
percent_change_7d             object
scrape_timestamp      datetime64[ns]
dtype: object

In [14]:
df.head(4)

,name,symbol,market_cap_usd,price_usd,circulating_supply,volume_24h_usd,percent_change_1h,percent_change_24h,percent_change_7d,scrape_timestamp
#,,,,,,,,,,
1,BTC Bitcoin,BTC,"$109,729,678,760",$6579.45,16677637,"$6,825,140,000",0.14%,5.64%,-9.47%,2017-11-13 10:08:47.538555
2,ETH Ethereum,ETH,"$30,201,865,132",$315.58,95702116,"$1,369,980,000",-0.11%,3.22%,4.57%,2017-11-13 10:08:47.538555
3,BCH Bitcoin Cash,BCH,"$21,126,209,247",$1257.49,16800300,"$4,594,130,000",7.16%,-18.35%,100.02%,2017-11-13 10:08:47.538555
4,XRP Ripple,XRP,"$7,783,447,925",$0.202002,"38,531,538,922 *","$160,219,000",-0.49%,1.67%,-2.11%,2017-11-13 10:08:47.538555


Notice that some columns have an asterix (used to denote that the currency are not minable).<br>
We can convert this feature into a new column by leveraging Pandas DataFrames' `apply` function -- <br>
which applies any function (anonymous or declared) across either columns (`axis`=0) or rows (`axis`=1).

In [15]:
def is_minable(row):
    '''
    Check if `circulating_supply` contains an asterix.
    This function operates on each row of the dataframe.
    If the ICO is not minable, we'll find an asterix and return 0.
    
    Note:
    That when we apply a function across a row,
    the entire row is treated as a key-value pair.
    '''
    circulating_supply = row['circulating_supply']
    
    if '*' in circulating_supply:
        return 0
    
    else:
        return 1

In [16]:
df['is_minable'] = df.apply(is_minable, axis=1)

In [17]:
df.head(4)

,name,symbol,market_cap_usd,price_usd,circulating_supply,volume_24h_usd,percent_change_1h,percent_change_24h,percent_change_7d,scrape_timestamp,is_minable
#,,,,,,,,,,,
1,BTC Bitcoin,BTC,"$109,729,678,760",$6579.45,16677637,"$6,825,140,000",0.14%,5.64%,-9.47%,2017-11-13 10:08:47.538555,1
2,ETH Ethereum,ETH,"$30,201,865,132",$315.58,95702116,"$1,369,980,000",-0.11%,3.22%,4.57%,2017-11-13 10:08:47.538555,1
3,BCH Bitcoin Cash,BCH,"$21,126,209,247",$1257.49,16800300,"$4,594,130,000",7.16%,-18.35%,100.02%,2017-11-13 10:08:47.538555,1
4,XRP Ripple,XRP,"$7,783,447,925",$0.202002,"38,531,538,922 *","$160,219,000",-0.49%,1.67%,-2.11%,2017-11-13 10:08:47.538555,0


### This table is now more human readible, but problematic for machines
Why? Because there are dollar signs, commas, asterix, and percent signs in numeric values.<br>
This causes most computers (and Pandas) to view numeric values as strings!

In [18]:
df.dtypes

name                          object
symbol                        object
market_cap_usd                object
price_usd                     object
circulating_supply            object
volume_24h_usd                object
percent_change_1h             object
percent_change_24h            object
percent_change_7d             object
scrape_timestamp      datetime64[ns]
is_minable                     int64
dtype: object

We can remove these symbols using regular expressions.<br>
Below is a dictionary of regular expressions we can use to weed out symbols

In [19]:
replace_symbols = {
    r'  [*]' : '',    # two spaces and any number of asterix
    r'[\$,%*]' : '',  # money signs, commas, percent signs, asterix
    r'[?]' : np.nan,  # question marks becomes a null value
    'Low Vol' : 0,    # low volume is simplified as zero...
}

Pandas `replace` operates on all all columns and all rows.<br>
The coolest aspect of this function is that 
1. it can take a dictionary as an input,
2. it can implement regular expressions, and
3. it can operate inplace

In [20]:
df.replace(replace_symbols, regex=True, inplace=True)

In [21]:
df.head(4)

,name,symbol,market_cap_usd,price_usd,circulating_supply,volume_24h_usd,percent_change_1h,percent_change_24h,percent_change_7d,scrape_timestamp,is_minable
#,,,,,,,,,,,
1,BTC Bitcoin,BTC,109729678760,6579.45,16677637,6825140000,0.14,5.64,-9.47,2017-11-13 10:08:47.538555,1
2,ETH Ethereum,ETH,30201865132,315.58,95702116,1369980000,-0.11,3.22,4.57,2017-11-13 10:08:47.538555,1
3,BCH Bitcoin Cash,BCH,21126209247,1257.49,16800300,4594130000,7.16,-18.35,100.02,2017-11-13 10:08:47.538555,1
4,XRP Ripple,XRP,7783447925,0.202002,38531538922,160219000,-0.49,1.67,-2.11,2017-11-13 10:08:47.538555,0


This looks good to me! Let's write this clean dataset to a csv.<br>
A best practice in data engineering is to create a function to programmatically generate file paths and directories.

In [22]:
def create_filename(root_dir, today):
    '''
    This function creates the filename, 
    it also creates the directory for the file if the directory doesn't exist.
    '''
    f_template = '{year}/{month}/{day}/{hour}/market_cap_USD_{time}.csv.gz'
    f = f_template.format(year = today.year,
                          month= today.month,
                          day  = today.day,
                          hour = today.strftime('%H'),
                          time = today.strftime('%H:%M:%S'))
    
    f_out = os.path.join(root_dir, f)
    
    dir_out = '/'.join(f_out.split('/')[:-1])
    if not os.path.exists(dir_out):
        os.makedirs(dir_out, exist_ok=True)
    
    return f_out

In [23]:
file = create_filename(root_dir, today)
file

'../2017/11/13/10/market_cap_USD_10:08:47.csv.gz'

In [24]:
df.to_csv(file, index=None, compression='gzip')

## Let's do some analysis <a id='analysis'></a>
How's the data look?

In [25]:
df = pd.read_csv(file, compression='gzip')

In [26]:
df.head()

,name,symbol,market_cap_usd,price_usd,circulating_supply,volume_24h_usd,percent_change_1h,percent_change_24h,percent_change_7d,scrape_timestamp,is_minable
0,BTC Bitcoin,BTC,1.097297e+11,6579.450000,1.667764e+07,6.825140e+09,0.14,5.64,-9.47,2017-11-13 10:08:47.538555,1
1,ETH Ethereum,ETH,3.020187e+10,315.580000,9.570212e+07,1.369980e+09,-0.11,3.22,4.57,2017-11-13 10:08:47.538555,1
2,BCH Bitcoin Cash,BCH,2.112621e+10,1257.490000,1.680030e+07,4.594130e+09,7.16,-18.35,100.02,2017-11-13 10:08:47.538555,1
3,XRP Ripple,XRP,7.783448e+09,0.202002,3.853154e+10,1.602190e+08,-0.49,1.67,-2.11,2017-11-13 10:08:47.538555,0
4,LTC Litecoin,LTC,3.273470e+09,60.830000,5.381156e+07,2.990170e+08,-0.58,2.94,9.49,2017-11-13 10:08:47.538555,1


Monetary and percentages are now floats!

In [27]:
df.dtypes

name                   object
symbol                 object
market_cap_usd        float64
price_usd             float64
circulating_supply    float64
volume_24h_usd        float64
percent_change_1h     float64
percent_change_24h    float64
percent_change_7d     float64
scrape_timestamp       object
is_minable              int64
dtype: object

There are aggregation functions we can use to calculate the market cap:

In [28]:
df['market_cap_usd'].sum()

203733779009.0

We can also find which ICOs have dropped by more than 60% this past week:

In [29]:
df_losers = df[df['percent_change_7d'] <= -60]
df_losers.head()

,name,symbol,market_cap_usd,price_usd,circulating_supply,volume_24h_usd,percent_change_1h,percent_change_24h,percent_change_7d,scrape_timestamp,is_minable
198,MCAP MCAP,MCAP,13900858.0,0.362988,3.829564e+07,738825.0,-1.72,-8.65,-71.40,2017-11-13 10:08:47.538555,0
424,NYC NewYorkCoin,NYC,1706768.0,0.000013,1.290328e+11,3755.0,-4.76,-2.96,-67.33,2017-11-13 10:08:47.538555,1
482,ARC Arcade Token,ARC,696507.0,0.385501,1.806758e+06,651.0,-0.30,-74.74,-80.25,2017-11-13 10:08:47.538555,0
483,NAUT NautilusCoin,NAUT,684071.0,0.042279,1.618000e+07,56766.0,-4.35,-0.25,-62.99,2017-11-13 10:08:47.538555,0
500,GRE Greencoin,GRE,550257.0,0.000156,3.528445e+09,1828.0,-1.87,-67.49,-92.65,2017-11-13 10:08:47.538555,0


Since we don't care about EVERY ICO, we can filter the dataframe by relevant symbols

In [30]:
watchlist = [
    'LTC',
    'BTC',
    'NEO'
]

In [31]:
df_w = df[df['symbol'].isin(watchlist)]
df_w

,name,symbol,market_cap_usd,price_usd,circulating_supply,volume_24h_usd,percent_change_1h,percent_change_24h,percent_change_7d,scrape_timestamp,is_minable
0,BTC Bitcoin,BTC,1.097297e+11,6579.45,16677637.0,6.825140e+09,0.14,5.64,-9.47,2017-11-13 10:08:47.538555,1
4,LTC Litecoin,LTC,3.273470e+09,60.83,53811557.0,2.990170e+08,-0.58,2.94,9.49,2017-11-13 10:08:47.538555,1
7,NEO NEO,NEO,1.817426e+09,27.96,65000000.0,4.652200e+07,-1.38,3.14,6.05,2017-11-13 10:08:47.538555,0


We can also calculate values in BTC

In [32]:
btc_price = df[df['symbol'] == 'BTC']['price_usd'].iloc[0]
btc_price

6579.4499999999998

In [33]:
df_w['price_btc'] = df_w['price_usd'] / btc_price

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [34]:
df_w[['name', 'symbol', 'price_usd', 'price_btc']].head()

,name,symbol,price_usd,price_btc
0,BTC Bitcoin,BTC,6579.45,1.000000
4,LTC Litecoin,LTC,60.83,0.009245
7,NEO NEO,NEO,27.96,0.004250


The data we just scraped is also [available in BTC](https://coinmarketcap.com/coins/views/all/#BTC), rather than USD.<br>
However, that table is rendered using Javascript, <br>
so it can't be scraped unless we use a client such as Selenium.

### One application is calculating your portfolio's worth

In [35]:
portfolio = [
    {'symbol' : 'BTC', 'quantity' : 2.045},
    {'symbol' : 'ETH', 'quantity' : 20.17},
    {'symbol' : 'DASH', 'quantity' : 1001},
]

In [38]:
def get_portfolio(df, portolio, cols=['name', 'symbol', 'price_usd', 
                                      'percent_change_24h', 
                                      'quantity', 'value_usd', 'value_btc']):
    '''
    Converts a dict `portfolio` into a pandas dataframe `df_p`,
    Inner join `df_p` to a dataframe of ICO prices (`df`).
    Calculates and printsvalue in USD and BTC.
    
    Retuns a dataframe with `cols` colums from the joined dataframe `df_`.
    '''
    df_p = pd.DataFrame(portfolio)
    btc_price = df[df['symbol'] == 'BTC']['price_usd'].iloc[0]
    
    df_ = df.merge(df_p, how='inner')
    df_['value_usd'] = df_['quantity'] * df_['price_usd']
    df_['value_btc'] = df_['value_usd'] / btc_price
    
    portfolio_usd = df_['value_usd'].sum()
    portfolio_btc = df_['value_usd'].sum() / btc_price
    
    print("Value of portfolio\n${:.2f} USD\n{:.4f} BTC".format(portfolio_usd, 
                                                               portfolio_btc))
    return df_[cols]

In [39]:
get_portfolio(df, portfolio)

Value of portfolio
$429659.65 USD
65.3033 BTC


,name,symbol,price_usd,percent_change_24h,quantity,value_usd,value_btc
0,BTC Bitcoin,BTC,6579.45,5.64,2.045,13454.97525,2.045000
1,ETH Ethereum,ETH,315.58,3.22,20.170,6365.24860,0.967444
2,DASH Dash,DASH,409.43,4.58,1001.000,409839.43000,62.290834


## Conclusions <a id='next-steps'></a>
Having programatic access to ICO prices is a first step for many applications.<br>
Please use this information responsibly!<br>
A lesson from [the 2010 Flash Crash](https://www.theguardian.com/business/2015/apr/22/2010-flash-crash-new-york-stock-exchange-unfolded).

Here are some next steps:
- Do this for BTC units.
- Host the data on s3 to another open source outlet with programmatic access.
- Host a cloud instance that generates this dataset in a regular interval.
- Analysis of of BTC's price on alt-coins.

I think there is some good software to be written.<br>
This wass done as a passion project during a cold Novemeber weekend,<br>
if this was helpful to you, any suggestions, time, or donations are appreciated!

Wallet locations for donations:
<a id='TODO'>todo</a>